### Getting top 10 Songs for an Artist
##### Testing this out so we can later apply to our data 
####  @author: Jyontika Kapoor

In [2]:
# pip install spotipy

### The below code uses the **spotipy** package to get the top tracks of an artist. Each artist has a spotify ID which we obtain using *sp.search()*. Then we get the top tracks through the *artist_top_tracks()* method. 

In [22]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from bs4 import BeautifulSoup

In [13]:
## CLIENT ID AND CLIENT SECRET
### DO NOT PUT ON GIT

client_id = 'spotify_client_id'
client_secret = 'spotify_client_secret'
genius_access_token = 'genius_access_token'


client_id = "c444fc8ca89845e1ae99dd2cfd698fb3"
client_secret = "acaad0ce292d442080348e6b9a46d9b3"
genius_access_token = "lOqXN-Ms2yerxQ84trh3ex5sV_QehWhhQNG2BecI0n3DcCuqV2Cq-Sb9py7U7ETO"


In [7]:

def get_top_tracks(artist_name, client_id, client_secret):
    # Initialize Spotify client with your credentials
    credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credentials)
    
    # search for the artist to get the Spotify ID through sp.search
    results = sp.search(q='artist:' + artist_name, type='artist') 
    items = results['artists']['items']
    if not items:
        return []
    
    
    artist_id = items[0]['id']
    
    top_tracks = sp.artist_top_tracks(artist_id)
    
    return [track['name'] for track in top_tracks['tracks']]


In [11]:
artist_name = 'Selena Gomez'
top_tracks = get_top_tracks(artist_name, client_id, client_secret)
print(top_tracks)

['Calm Down (with Selena Gomez)', 'Lose You To Love Me', 'Single Soon', 'Love On', 'It Ain’t Me (with Selena Gomez)', 'People You Know', 'Wolves', 'Back To You - From 13 Reasons Why – Season 2 Soundtrack', 'Fetish (feat. Gucci Mane)', 'Same Old Love']


#### Now we want to get the lyrics for each of the 10 tracks. We will do this using the **genius API** and the requests & bs4 library. The idea is we will search a song name on Genius through the requests library, find a match, and use bs4 to parse the HTML and extract the lyrics.

In [30]:
def get_lyrics(song_name, artist_name, genius_access_token):
    #set up the request using access token 
    base_url = "https://api.genius.com"
    headers = {'Authorization': 'Bearer ' + genius_access_token}
    search_url = base_url + "/search" #getting ready to search!
    data = {'q': song_name} #we want to search the song name!

    response = requests.get(search_url, params=data, headers=headers)
    json = response.json()
    path = None

    # loops through the search results returned by the Genius API to find a specific song that matches name of song and artist name

    for hit in json["response"]["hits"]:
        if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
            path = hit["result"]["path"]
            break #assuming the first is right 

    if path:
        lyrics_url = "https://genius.com" + path
        lyrics_response = requests.get(lyrics_url)
        soup = BeautifulSoup(lyrics_response.text, 'lxml')
        
        # used "inspect" element to find where lyrics are in HTML 
        lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
        
        # extract the text ... if lyrics are not found return lyrics not found
        lyrics = lyrics_div.get_text(separator='\n', strip=True) if lyrics_div else "Lyrics not found"
        return lyrics
    
    
    else:
        return "Lyrics not found"


In [31]:
artist_name = 'Kesha'
tracks = get_top_tracks(artist_name, client_id, client_secret)

In [32]:
lyrics = get_lyrics(tracks[1], artist_name, genius_access_token)
print(lyrics)

[Verse 1]
Wake up in the morning feelin' like P. Diddy
(
Hey, what up, girl?
)
Grab my glasses, I'm out the door, I'm gonna hit this city
(
Let's go
)
Before I leave, brush my teeth with a bottle of Jack
'Cause when I leave for the night, I ain't coming back
[Pre-Chorus]
I'm talkin' pedicure on our toes, toes, tryin' on all our clothes, clothes
Boys blowin' up our phones, phones
Drop-toppin', playin' our favorite CDs, pullin' up to the parties
Tryna get a little bit tipsy
[Chorus]
Don't stop, make it pop, DJ, blow my speakers up
Tonight, I'ma fight 'til we see the sunlight
Tick tock on the clock, but the party don't stop, no
Oh, woah, woah, oh, oh, woah, woah, oh
Don't stop, make it pop, DJ, blow my speakers up
Tonight, I'ma fight 'til we see the sunlight
Tick tock on the clock, but the party don't stop, no
Oh, woah, woah, oh, oh, woah, woah, oh
[Verse 2]
Ain't got a care in the world, but got plenty of beer
Ain't got no money in my pocket, but I'm already here
And now the dudes are li